GELİŞTİRMELER--

-İlk iyileştirme olarak, datasetimizdeki filmlerin clean kısmındaki kelimelerimizin büyük ve küçük harfler içerdiğini tespit ettim. Bu durum algoritmada 2 kelime aynı kelime olsa bile birisi büyük harfle başladığı için farklılığa sebep olabileceğinden dolayı bütün kelimeleri küçük harfe çevirerek daha doğru bir sonuç elde etmeye çalıştım.

-1 ve 2 harfli kelimeleri stopwords saydığım için bu kelimeleri yok ettim. Kelimeleri yok etmemin sebebi 1 veya 2 harfli olan kelimelerin çok fazla ortak filmde çıkması ve bu kelimelerin ayırt edici bir özelliği olmadığıydı. Bu kelimeleri elekten geçirerek kelime havuzunu daha anlamlı hale getirmeye çalıştım.

-Kod içerisinde doc frekanslarını sıralayarak incelediğimde bazı kelimelerin anlam ifade etmemesine rağmen çok fazla frekansa sahip olduğunu gördüm. Bu yüzden bir frekans eşiği koymaya karar verdim. Çok sayıda filmde ortak geçen kelimeler istediğimiz sonucu almamızda yanıltıcı oluyorlardı. En çok frekansa sahip olan kelimelerden bazıları şunlardı = film, olmak, ada, bir, ama, etmek, son, yaş, mak, mek, ken .Bu gibi kelimeler çok sayıda filmde geçiyor ve anlamsızlar. Verceğim eşik değerini belirlemek için her filmin clean datasına dahil ettiğimiz türlerin ne kadar frekansa sahip olduklarına bakarak karar verdim. Bakmış olduğum sonuçlara göre 5261 filmin bulunduğu datasetindeki tür frekansları büyükten küçüğe şu şekildeydi  "dram" -> 2353 , "Komedi" -> 1471,   Aksiyon -> 980 ve Macera -> 976 şeklindeydi. Bu sonuçları göz önünde bulundururak film sayısının yarısını eşik değeri olarak belirledim. Frekensı film sayısının yarısından büyük olan kelimeleri elekten geçirdim ve kelimelerin doc frekanslarını 0 a eşitledim.

-Clean verilerimiz içerisine filmin adını ve yönetmenlerini de eklemenin benzer filmler çıkmasına katkıda bulunacağını düşünerek bu verileri de clean dataya eklemeye karar verdim. Ancak yönetmen isimlerinin ve soyisimlerinin bir kalıp olarak alınmasının daha doğru sonuçlar çıkaracağını düşündüm çünkü iki filmin yönetmeninin ismi veya soyismi aynı olursa bu filmlere algoritma benzer diyebilirdi. Bu yüzden yönetmenlerin ismini birleştirerek clean dataya eklemeyi tercih ettim. Örnek= Yönetmen ->joe russo, clean dataya eklendiği şekil ->joerusso . 

-Eski algoritmayı incelediğimde tf değeri için sadece kelimenin dökümanda geçme sayısı alınmış fakat yaptığım araştırmadan tf değerinin, kelimenin dökümanda geçme sayısı / dökümandaki toplam kelime sayısı formülüyle hesaplandığını gördüm. Bu yüzden for döngüsü kullanarak her bir filmin clean datasının içerisinde kaç kelime olduğunu hesapladım ve bunu bir liste olarak tuttum. Daha sonra tf i hesaplarken her filmin clean datasındaki toplam kelime sayısını bölen olarak kullandım. Tf i bu şekilde hesaplamazsak içerisinde fazla kelime olan clean daha büyük ihtimalle daha fazla önerilecektir. Fakat tf i hesaplarken o filmin clean datasındaki kelime sayısına bölersek daha adaletli bir tf değeri oluşturmuş olduğumuzu düşünüyorum.

-İdf hesabının yapıldığı kısımda eklenmiş olan (+1) değerlerinin idf-tf değerini olumsuz etkilediğini düşündüm. Çünkü doc frekansı düşük olan veya 0 olan bir kelime geldiğinde biz bu kelimeye +1 ekleyerek idf değerini yükseltmiş oluyoruz. +1 değerlerini kaldırdığımda 0 gelen frekanslar olduğu için ZeroDivisionError hatası aldım. Bu yüzden bir try catch yapısı kullanarak catch kısmına düşüldüğünde idf değerini 0 a eşitleyerek bu sorunu giderdim.

Özet: 
Bu geliştirmeleri yaptıktan sonra eski ve yeni algoritmaya aynı filmleri göndererek iki algoritma arasındaki farkı ölçmeye çalıştım. Yapmış olduğum testlerde yeni algoritmanın eski algoritmaya göre daha iyi sonuçlar verdiğini gözlemledim. Geliştirilmiş algoritmanın kötü yanıysa eski algoritmaya göre biraz daha yavaş çalışmasıydı. Bunun en büyük sebebi de iç içe for döngüsü kullanarak her filmdeki kelimeleri saydırmamdı. 


In [103]:
import pandas as pd
import numpy as np
import re 

In [104]:
df_diskten = pd.read_excel("filmler_clean.xlsx")

In [146]:
df_diskten.head()

,film adi,ozet ve detaylar,tur,oyuncular,yonetmen,vizyon tarihi,sure,clean
0,Bergen,Arabeskin unutulmayan kadın seslerinden biri o...,"['Dram', 'Biyografik']","['Farah Zeynep Abdullah', 'Erdal Beşikçioğlu',...","['Mehmet Binay', 'Caner Alper']",4 Mart 2022,2s 26dk,arabesk unutmak kadın ses o Bergen inmek hayat...
1,Kirpi Sonic 2,Çocuk filmi \n\n\nKirpi Sonic filminin devam h...,"['Animasyon', 'Aile', 'Macera']","['Jim Carrey', 'James Marsden', None]",['Jeff Fowler'],8 Nisan 2022,2s 2dk,çocuk film kirpi Sonic film deva halka o yapım...
2,Fantastik Canavarlar: Dumbledore'un Sırları,"Karanlık büyücü Gellert Grindelwald, büyücülük...","['Fantastik', 'Macera']","['Eddie Redmayne', 'Jude Law', 'Mads Mikkelsen']",['David Yates'],15 Nisan 2022,2s 22dk,Karanlık büyü Gellert Grindelwald büyü Dünya k...
3,Aykut Enişte 2,"Aykut Enişte 2, sevdiği kadınla evlilik arifes...",['Komedi'],"['Cem Gelinoğlu', 'Melis Babadağ', 'Hakan Yılm...",['Onur Bilgetay'],3 Aralık 2021,1s 55dk,Aykut enişte 2 sevmek kadın ev arife kendi tür...
4,Morbius,"Morbius, hastalığına deva bulmaya çalışırken b...","['Aksiyon', 'Fantastik', 'Macera']","['Jared Leto', 'Matt Smith (XI)', 'Adria Arjona']",['Daniel Espinosa'],1 Nisan 2022,1s 45dk,Morbius hastalık deva bulmak Çalışır bambaşka ...


In [148]:
clean = df_diskten["clean"]
len(clean)

5261

In [105]:
kelime2index = {}
index2kelime = {}

In [106]:
data_set = df_diskten["clean"]
lwr_ds=[x.lower() for x in data_set] #bütün kelimeleri küçük harfe çeviriyorum
clean_col=[re.sub(r'\b\w{1,2}\b', '', m) for m in lwr_ds] #uzunluğu 1 ve 2 olan kelimeleri siliyorum

In [107]:
i = 0
for cell in clean_col:
    
    words = cell.split()
    
    
    for word in words: #burada içerisinde bütün kelimelerin olduğu bir sözlük oluşturuluyor
        if word in kelime2index:
            continue
        kelime2index[word] = i
        index2kelime[i] = word
        i += 1


In [108]:
film_sayisi = len(clean_col)
vocab_size = len(kelime2index)
filmler = np.zeros((film_sayisi, vocab_size)) 

In [109]:
for i, cell in enumerate(clean_col): #hangi kelimenin hangi filmde kaç kere geçtiğini filmler matrisine yazıyor
    words = cell.split()
    for word in words:
        wordIndex = kelime2index[word]
        filmler[i][wordIndex] += 1


In [110]:
df_deleted=[] 
doc_frequencies = {}
l=0
for key,word in enumerate(kelime2index):  #belge sıklığı hesaplanıyor yani bir kelimenin kaç farklı filmde geçtiği
    doc_count = 0
    for film in clean_col:
        if word in film:
            doc_count += 1
    if(doc_count<(len(clean_col)/2)): #eğer doc frekansı film sayısının yarısından büyükse bunu stopwords olarak sayıp bu kelimeleri eliyorum
        doc_frequencies[word] = doc_count
    else:
        df_deleted.append(word) #eğer stopwords ise doc frekansını 0 yapıp df_deleted listesine ekle
        doc_frequencies[word]=0
        for l in range(film_sayisi): #stopwordlere filmler matrisinde 0 değerini veriyorum
            filmler[l][key-1]=0
        


In [111]:
sorted_frequencies = sorted(doc_frequencies, key=doc_frequencies.get, reverse=True)

In [149]:
for word in sorted_frequencies[:10]:
    print(word, doc_frequencies[word])

end 2552
yap 2497
yön 2489
yat 2456
yan 2406
dram 2357
ile 2331
kendi 2316
ark 2274
hayat 2244


In [112]:
filmler_tf_idf = np.zeros((film_sayisi, vocab_size))

In [113]:
kelime_sayaci=0     
film_kelime_sayaci=[] 
#Her film için dökümanının içerisinde kaç kelime olduğunu buluyorum. Bunu yapma sebebim kelimenin dökümanda geçme sayısını,dökümandaki toplam kelime sayısına bölerek tf değerini bulmak
for x,film in enumerate(filmler):
    for y,tf in enumerate(film):
        kelime_sayaci=kelime_sayaci+filmler[x][y]
    film_kelime_sayaci.append(kelime_sayaci)
    kelime_sayaci=0
        

In [130]:
for i, film in enumerate(filmler):
    if i % 1000 == 0:
        print(f"{i} inci film işleniyor...")
    for j, z in enumerate(film):
        idf = 0
        tf=z/film_kelime_sayaci[i] #kelimenin dökümanda geçme sayısını, o dökümandaki toplam kelime sayısına bölerek tf değerini buluyorum
        if tf > 0:
            
            try:
              idf = np.log(film_sayisi  / (doc_frequencies[index2kelime[j]] )) #idf değeri hesaplanıyor yani df değerinin logaritması alınıyor
            except ZeroDivisionError:
              idf = 0
        tf_idf_score = tf * idf  #idf ve tf değerleri çarpılarak filmler_tf_idf matrisi oluşturuluyor bu matriste her kelimenin bir skoru oluyor
        filmler_tf_idf[i][j] = tf_idf_score



0 inci film işleniyor...
1000 inci film işleniyor...
2000 inci film işleniyor...
3000 inci film işleniyor...
4000 inci film işleniyor...
5000 inci film işleniyor...


In [131]:
def cos_sim(v1, v2):
    return np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))

In [132]:
def film_2_str(row_index):
    film = df_diskten.iloc[row_index]
    adi = film['film adi']
    turu = film['tur']
    ozet = film['ozet ve detaylar']
    if len(ozet) > 150:
        ozet = ozet[:150] + "..."

    return f"Film adı: {adi} \nTürü: {turu} \nÖzeti: {ozet}"


In [161]:
print(film_2_str(50))

Film adı: Zindan Adası 
Türü: ['Gerilim'] 
Özeti: Zindan Adası, Martin Scorsese'nin bir çok filmi gibi yine bir başyapıt statüsünde. Filmde, Teddy Daniels ve Chuck Aule isimli iki polis memurunun, Rac...


In [133]:
def most_similar_films(film_adi, benzer_film_sayisi):
    film_indexi = df_diskten[df_diskten["film adi"] == film_adi].index[0]
    print("Aranan Film:")
    print(film_2_str(film_indexi))
    print()
    print("* " * 30)
    print()

    benzerlikler = []
    sorgu_film = filmler_tf_idf[film_indexi]
    for vec in filmler_tf_idf:
        benzerlik = cos_sim(sorgu_film, vec)
        benzerlikler.append(benzerlik)
    
    en_benzer_film_indexleri = (- np.array(benzerlikler)).argsort()[:benzer_film_sayisi + 1]
    print("Benzer Filmler:\n")
    for i, film_idx in enumerate(en_benzer_film_indexleri[1:]):
        print(str(i+1) + ") " + film_2_str(film_idx))
        print()
        print("- " * 20)
        print()

In [144]:
most_similar_films("Hızlı ve Öfkeli", 5)

Aranan Film:
Film adı: Hızlı ve Öfkeli 
Türü: ['Aksiyon', 'Polisiye'] 
Özeti: Hızlı ve Öfkeli'de Domenic Toretto, arabaların pahalı aksesuarlarını çalan bir çetenin baş üyesidir. Çok değerli elektronik ekipmanların çalınmış olma...

* * * * * * * * * * * * * * * * * * * * * * * * * * * * * * 

Benzer Filmler:

1) Film adı: Hızlı ve Öfkeli 9 
Türü: ['Aksiyon'] 
Özeti: Hızlı ve Öfkeli 9, geçmişinden gelen bir tehdide karşı mücadele etmek zorunda kalan Dom ve ekibinin hikayesini konu ediyor. Dominic Toretto’nun artık ...

- - - - - - - - - - - - - - - - - - - - 

2) Film adı: Hızlı ve Öfkeli 5: Rio Soygunu 
Türü: ['Aksiyon', 'Gerilim'] 
Özeti: 2001'de başladığından beri devamını getirmek konusunda hızı ve öfkesi dinmeyen seri 'Hızlı ve Öfkeli'nin ekibe tekrar katılan Dwayne Johnson'lı son ma...

- - - - - - - - - - - - - - - - - - - - 

3) Film adı: Hızlı ve Öfkeli 4 
Türü: ['Aksiyon'] 
Özeti: Hızlı ve Öfkeli 4'te düzenbazlıktan "emekli" Dominic Toretto'nun (Vin Diesel) Meksika sınırını a